In [1]:
import pandas as pd
import duckdb

# Data

In [2]:
dpt_to_region_mapping = {
    "59":"HDF",
    "62":"HDF",
    "75":"IDF",
    "95":"IDF",
    "83":"PACA",
    "84":"PACA",
}

dpts_dfs = []
for year in range(2016,2022):
    # https://www.insee.fr/fr/statistiques/1893198
    df = pd.read_excel("data/estim-pop-dep-sexe-gca-1975-2023.xls", str(year), skiprows=3)
    subset = df[df["Départements"].isin(["59", "62", "75", "95", "83", "84"])][["Départements", "Unnamed: 7"]]
    subset["year"] = year
    dpts_dfs.append(subset)

dpts_dfs = pd.concat(dpts_dfs)
dpts_dfs.columns = ["departement", "population", "year"]
dpts_dfs["region"] = dpts_dfs["departement"].apply(lambda x: dpt_to_region_mapping.get(x))
dpts_dfs = dpts_dfs[["region", "departement", "year", "population"]]
dpts_dfs.head()

,region,departement,year,population
60,HDF,59,2016,2603723
63,HDF,62,2016,1470725
76,IDF,75,2016,2190327
84,PACA,83,2016,1055821
85,PACA,84,2016,559014


# Rollup

In [3]:
query = """
SELECT year, region, departement, 
SUM(population),
FROM dpts_dfs
GROUP BY 
GROUPING SETS ((year, region, departement), (year, region), year)
ORDER BY year, region, departement
"""
duckdb.sql(query).df().head(11)

,year,region,departement,sum(population)
0,2016,HDF,59,2603723.0
1,2016,HDF,62,1470725.0
2,2016,HDF,None,4074448.0
3,2016,IDF,75,2190327.0
4,2016,IDF,95,1221923.0
5,2016,IDF,None,3412250.0
6,2016,PACA,83,1055821.0
7,2016,PACA,84,559014.0
8,2016,PACA,None,1614835.0
9,2016,None,None,9101533.0


### Même chose que le grouping sets ci dessus, avec ROLLUP

En lisant chaque ligne, on constate que ROLLUP calcule le total:
- d'abord pour chaque département d'une région
- ensuite pour cette région
- puis les départements d'une autre région
- et le total pour cette région
- ...
- puis le total de l'ensemble des régions, pour l'année 2016
- puis on reprends à 2017, etc...


In [5]:
query = """
SELECT year, region, departement, 
SUM(population),
FROM dpts_dfs
GROUP BY ROLLUP (year, region, departement)
HAVING year NOT NULL
ORDER BY year, region, departement
"""
duckdb.sql(query).df()

,year,region,departement,sum(population)
0,2016,HDF,59,2603723.0
1,2016,HDF,62,1470725.0
2,2016,HDF,None,4074448.0
3,2016,IDF,75,2190327.0
4,2016,IDF,95,1221923.0
5,2016,IDF,None,3412250.0
6,2016,PACA,83,1055821.0
7,2016,PACA,84,559014.0
8,2016,PACA,None,1614835.0
9,2016,None,None,9101533.0


<b> Attention </b> 

Notez qu'on utilise "HAVING year NOT NULL" <br />
Pour éviter d'avoir la somme globale des populations toutes années confondues <br />
(ce qui n'a pas de sens)

#### Explications

Rollup fait les aggrégations en enlevant systématiquement <br />
la catégorie la plus à droite:

ROLLUP (year, region, departement):
- agrégation 1: (year, region, departement)
- agrégation 2: (year, region)
- agrégation 3: (year)
- agrégation 4: ensemble

Vous n'aurez donc pas la combinaison year / département, ni region seule ou département seul...

Mais pour ça, il y a CUBE ;)

# Cube

Avec CUBE, on a toutes les combinaisons possibles:
- year, region, departement
- year, region,
- year, departement
- departement, region
- year, 
- region,
- departement,
- ensemble



=== >

In [6]:
query = """
SELECT year, region, departement, 
SUM(population),
FROM dpts_dfs
GROUP BY CUBE (year, region, departement)
-- HAVING year NOT NULL
ORDER BY year, region, departement
"""
duckdb.sql(query).df()

,year,region,departement,sum(population)
0,2016.0,HDF,59,2603723.0
1,2016.0,HDF,62,1470725.0
2,2016.0,HDF,None,4074448.0
3,2016.0,IDF,75,2190327.0
4,2016.0,IDF,95,1221923.0
...,...,...,...,...
107,NaN,None,75,12995887.0
108,NaN,None,83,6438285.0
109,NaN,None,84,3364396.0
110,NaN,None,95,7449117.0


Vérification: 

In [7]:
dpts_dfs.groupby(["year", "region", "departement"])["population"].sum().shape[0] + \
dpts_dfs.groupby(["year", "region"])["population"].sum().shape[0] + \
dpts_dfs.groupby(["year", "departement"])["population"].sum().shape[0] + \
dpts_dfs.groupby(["departement", "region"])["population"].sum().shape[0] + \
dpts_dfs.groupby(["year"])["population"].sum().shape[0] + \
dpts_dfs.groupby(["region"])["population"].sum().shape[0] + \
dpts_dfs.groupby(["departement"])["population"].sum().shape[0] + \
1  # dpts_dfs["population"].sum() => Un seul chiffre

112

Dans l'exemple de la population des regions, ça n'est pas tellement intéressant:
- ne pas grouper par an n'a aucun sens: on additionne plusieurs fois la même population
- ne pas grouper par région n'a pas d'intérêt: aucun habitant d'un département n'habite dans plusieurs régions

Mais dans le cadre de ventes d'une entreprise:
- grouper sans l'année donne les ventes totales historiques
- grouper sans le commercial permet d'avoir les totaux par région
- un commercial peut avoir travaillé dans plusieurs régions (mutation)


ROLLUP et CUBE sont donc des clauses dont vous pourriez avoir besoin à l'occasion ;)